**Compare protocol SYN_BACKGROUND_HIGH**

Run the protocol SYN_BACKGROUND_HIGH with the full Gillies & Willshaw (2005) STN model, then reduce the model according to our BranchZip algorithm, then rerun protocol and compare responses

# README

- For each synapse, the median frequency of a stereotipical PSP is measured (in cell with spiking Na channels disabled)
- Synapses are still mapped to a location where the transfer impedance Zc is closest to that in original model
- However, for each synapse type, Zc is computed at the aforementioned median frequency
- => this should make the PSP at the soma elicited by that synapse in closer correspondence to the full model

# Imports

In [ ]:
# Enable interactive plots with backend 'notebook'
%matplotlib notebook

# print code version (hash of checked out version)
!git log -1

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..

In [ ]:
# Import all our functions
from evalmodel.stn_model_evaluation import *

# Full STN model

Build the Gillies & Willshaw STN model

In [ ]:
full_model = StnModel.Gillies2005
red_model = StnModel.Gillies_FoldBush_Tapered

# Make cell model and evaluator
evaluator = StnModelEvaluator(full_model, PhysioState.NORMAL)
evaluator.build_cell(full_model)

In [ ]:
# Make all stimuli/synapses for stimulation protocol
proto = StimProtocol.SYN_BACKGROUND_HIGH
evaluator._setup_proto(proto)

In [ ]:
# inspect the model in NEURON GUI (this pops up a GUI window outside browser)
# click __Tools > ModelView > Soma > Point Processes > GABAsyn & GLUsyn
# from neuron import gui

import matplotlib.pyplot as plt
import bgcellmodels.morphology.morph_gui as morphology
from mpl_toolkits.mplot3d import Axes3D

plt.figure()
ax = plt.subplot(111, projection='3d')
ax.view_init(90,90) # set azimut and elevation
ax.set_zlim((-1,1)) # prevent rounding errors
ax.set_axis_off() # hide x,y,z axes and grid

# Plot shape of our cell
h.define_shape()
morphology.shapeplot(h, ax)

# Mark locations of inputs
all_inputs = evaluator.merged_inputs(['gpe','ctx'], evaluator.target_model)
marker_colors = 'rgby'
for i, syn in enumerate(all_inputs['synapses']):
    morphology.mark_locations(h, 
                            syn.get_segment().sec,
                            syn.get_loc())

## Run protocol


Explanation of plots:

- the first plot are incoming spikes from GPE (i.e.: spikes generated with a temporal pattern that is statistically similar to that of GPe neurons in vivo)


- the second plot are incoming spikes from Cortex (CTX)


- the first plot is the membrane voltage in several compartments (soma, a radnom distal segment, and in several compartments where GLU and GABA synapses are located)

In [ ]:
# Run protocol
evaluator._run_proto(proto)
evaluator.plot_traces(proto)

# Reduced STN model

Collapse dendritic trees using BranchZip algorithm, then remap synapses

In [ ]:
# red_model = StnModel.Gillies_FoldBush_Tapered
red_model = StnModel.Gillies_FoldMarasco_Legacy

# Model reduction
evaluator.build_cell(red_model)
evaluator.target_model = red_model

## Plot morphology & synapses

Run the same stimulation protocol with the reduced model.

In [ ]:
# Run Protocol
evaluator._setup_proto(proto)

# import matplotlib.pyplot as plt
# import bgcellmodels.morphology.morph_gui as morphology
# from mpl_toolkits.mplot3d import Axes3D

plt.figure()
ax = plt.subplot(111, projection='3d')
ax.view_init(90,90) # set azimut and elevation
ax.set_zlim((-1,1)) # prevent rounding errors
ax.set_axis_off() # hide x,y,z axes and grid

# Plot shape of our cell
h.define_shape()
morphology.shapeplot(h, ax)

# Mark locations of inputs
all_inputs = evaluator.get_all_inputs(red_model)
all_synapses = all_inputs['synapses']
print("Mapped {} synapses".format(len(all_synapses)))

marker_colors = 'rgby'
for i, syn in enumerate(all_synapses):
    morphology.mark_locations(h, 
                            syn.get_segment().sec,
                            syn.get_loc())

## Run protocol

In [ ]:
evaluator._run_proto(proto)
evaluator.plot_traces(proto)